In [9]:
%cd ../../..

/home/users/dmoreno2016/ASTROMER/astromer_pe_main/astromer


In [ ]:
import os

path_logs = './presentation/experiments/astromer_1_pe/results/pretraining/P02R01/exp_003_b/logs/train'
nlog=-1

train_logs = [x for x in os.listdir(path_logs) if x.endswith('.v2')][nlog]
path_train = os.path.join(path_logs, train_logs)
path_train


In [2]:
from src.models.astromer_1 import get_ASTROMER

import tensorflow as tf
import numpy as np
import os
import toml

2023-10-31 19:46:52.352548: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-31 19:46:53.114623: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [4]:
def cargar_modelo(ruta, model_config, pe_config, data_name):
    """Carga un modelo guardado en la ruta especificada."""
    astromer = get_ASTROMER(num_layers=model_config['num_layers'],
							num_heads=model_config['num_heads'],
							head_dim=model_config['head_dim'],
							mixer_size=model_config['mixer'],
							dropout=model_config['dropout'],
							pe_type=model_config['pe_type'],
							pe_config=pe_config,
                            pe_func_name=model_config['pe_func_name'],
                            residual_type=model_config['residual_type'],
							window_size=model_config['window_size'],
							encoder_mode=model_config['encoder_mode'],
							average_layers=model_config['avg_layers'],
							data_name=data_name)

    astromer.load_weights(os.path.join(ruta, 'weights', 'weights'))

    return astromer

def comparar_pesos(modelo1, modelo2):
    """Compara los pesos de dos modelos y devuelve la diferencia."""
    pesos1 = modelo1.get_weights()
    pesos2 = modelo2.get_weights()

    if len(pesos1) != len(pesos2):
        raise ValueError("Los modelos no tienen la misma cantidad de capas/pesos.")

    diferencias = []
    for w1, w2 in zip(pesos1, pesos2):
        diferencia = np.mean(np.abs(w1 - w2))
        diferencias.append(diferencia)

    return diferencias

# Rutas a los modelos
ruta_cfg = 'presentation/experiments/astromer_1_pe/results/pretraining/P02R01/pretrained_weights'
ruta_modelo1 = 'presentation/experiments/astromer_1_pe/results/pretraining/P02R01/pretrained_weights'
#ruta_modelo1 = 'presentation/experiments/astromer_1_pe/results/finetuning/P02R01_kepler/pretrained_weights/lr_1e-4_pe_by_mean/nontrain/kepler/fold_0/kepler_50'
ruta_modelo2 = 'presentation/experiments/astromer_1_pe/results/finetuning/P02R01_kepler/pretrained_weights/lr_1e-3_pe_by_mean/PE/kepler/fold_0/kepler_50'
#ruta_modelo2 = 'presentation/experiments/astromer_1_pe/results/finetuning/P02R01/pretrained_weights/lr_scheduler/PE/alcock/fold_0/alcock_50'
data_name = 'kepler'	

with open(os.path.join(ruta_cfg, 'config.toml'), 'r') as f:
    model_config = toml.load(f)

if 'residual_type' not in model_config:
    model_config['residual_type'] = None

with open('{}/pe_config.toml'.format(ruta_cfg), mode="r") as fp:
    pe_config = toml.load(fp)

# Cargar modelos
modelo1 = cargar_modelo(ruta_modelo1, model_config, pe_config, data_name)
modelo2 = cargar_modelo(ruta_modelo2, model_config, pe_config, data_name)

# Comparar pesos
diferencias = comparar_pesos(modelo1, modelo2)

# Imprimir diferencias
for i, diferencia in enumerate(diferencias):
    print(f"Capa {i + 1}: Diferencia media en pesos = {diferencia}")

# Imprimir diferencias junto con los nombres de las capas
# for i, (capa, diferencia) in enumerate(zip(modelo1.layers, diferencias)):
#     print(f"Capa ({i + 1}) '{capa.name}': Diferencia media en pesos = {diferencia:.5f}")

asasasas
kepler
We are using kepler data
Using masking...


Using masking...


In [14]:
for i, (capa, diferencia) in enumerate(zip(modelo1.get_layer('encoder').layers, diferencias)):
    print(f"Capa ({i + 1}) '{capa.name}': Diferencia media en pesos = {diferencia:.5f}")

Capa (1) 'inp_transform': Diferencia media en pesos = 0.00000
Capa (2) 'PosEncoding': Diferencia media en pesos = 0.00000
Capa (3) 'att_layer_0': Diferencia media en pesos = 0.00447
Capa (4) 'att_layer_1': Diferencia media en pesos = 0.00000
Capa (5) 'dropout_4': Diferencia media en pesos = 0.00000


In [16]:
len(modelo1.get_weights())

39

In [17]:
layers_model_1 = modelo1.weights
layers_model_2 = modelo2.weights

print('LAYERS COMPARISON:')
for layer_1, layer_2 in zip(layers_model_1, layers_model_2):
    aux = 'Different'
    
    if np.sum(layer_1.numpy() != layer_2.numpy()) == 0:
        aux = 'Same'
        
    print(f'Weights are {aux} --> Name Layers: Model 1 ({layer_1.name}) - Model 2 ({layer_2.name}) \n')   

LAYERS COMPARISON:
Weights are Same --> Name Layers: Model 1 (encoder/inp_transform/kernel:0) - Model 2 (encoder/inp_transform/kernel:0) 

Weights are Same --> Name Layers: Model 1 (encoder/inp_transform/bias:0) - Model 2 (encoder/inp_transform/bias:0) 

Weights are Different --> Name Layers: Model 1 (encoder/PosEncoding/pos_emb:0) - Model 2 (encoder/PosEncoding/pos_emb:0) 

Weights are Same --> Name Layers: Model 1 (encoder/att_layer_0/head_attention_multi/WQ/kernel:0) - Model 2 (encoder/att_layer_0/head_attention_multi_2/WQ/kernel:0) 

Weights are Same --> Name Layers: Model 1 (encoder/att_layer_0/head_attention_multi/WQ/bias:0) - Model 2 (encoder/att_layer_0/head_attention_multi_2/WQ/bias:0) 

Weights are Same --> Name Layers: Model 1 (encoder/att_layer_0/head_attention_multi/WK/kernel:0) - Model 2 (encoder/att_layer_0/head_attention_multi_2/WK/kernel:0) 

Weights are Same --> Name Layers: Model 1 (encoder/att_layer_0/head_attention_multi/WK/bias:0) - Model 2 (encoder/att_layer_0/he

In [6]:
def print_layers(model, prefix=''):
    for layer in model.layers:
        print(prefix + layer.name)
        if hasattr(layer, 'layers') and len(layer.layers) > 0:
            print_layers(layer, prefix=prefix + '  ')

print_layers(modelo1)

mask
input
times
encoder
  inp_transform
  PosEncoding
  att_layer_0
  att_layer_1
  dropout_4
regression


In [8]:
def print_weights_and_names(model, prefix=''):
    for layer in model.layers:
        weights = layer.get_weights()
        if weights:
            print(prefix + layer.name)
            for w in weights:
                print(w)
                print('---')
        
        # Si la capa actual tiene subcapas, revisamos esas también
        if hasattr(layer, 'layers') and len(layer.layers) > 0:
            print_weights_and_names(layer, prefix=prefix + '  ')
            
print_weights_and_names(modelo1)

encoder
[[ 1.25655219e-01 -1.49459988e-01 -1.44042611e-01  1.30089700e-01
   1.71717461e-02  1.10155409e-02  2.09142163e-01  1.72458962e-01
   1.63776770e-01  5.63616157e-02  2.19869435e-01  6.68852031e-02
  -9.50858518e-02  1.11473560e-01  1.82494476e-01  1.49523467e-01
   1.60315946e-01 -7.20822215e-02  1.14447296e-01  2.29373220e-02
   1.19285770e-01 -3.78774405e-01  2.84548216e-02 -1.67331338e-01
  -1.57028526e-01  1.67449206e-01  1.51335940e-01 -5.22844046e-02
   5.87115623e-02 -5.84900640e-02  2.39014313e-01  6.18554913e-02
  -1.69046611e-01  6.38524489e-03 -2.68956661e-01  1.52485847e-01
  -2.74250865e-01  1.05571732e-01  1.72876820e-01  2.15249225e-01
  -1.98346749e-01 -1.71647221e-01  1.68198660e-01  1.69854313e-01
  -2.06739724e-01 -1.51502594e-01  1.61653534e-01  1.98836118e-01
   1.39751583e-01  6.04324192e-02  1.57046616e-01 -3.12844902e-01
  -2.86209613e-01 -3.06806535e-01 -2.30959460e-01 -1.84469685e-01
  -1.18328042e-01 -5.53404465e-02 -2.62152970e-01 -3.16709310e-01
  

In [ ]:
print(á)